In [1]:
!wget -c https://alphacephei.com/vosk-colab/kaldi.tar.gz
!tar xzf kaldi.tar.gz

--2022-08-26 06:59:22--  https://alphacephei.com/vosk-colab/kaldi.tar.gz
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 809190128 (772M) [application/octet-stream]
Saving to: ‘kaldi.tar.gz’

kaldi.tar.gz        100%[===================>] 771.70M  11.4MB/s    in 72s     

2022-08-26 07:00:35 (10.7 MB/s) - ‘kaldi.tar.gz’ saved [809190128/809190128]



In [2]:
%cd /content/kaldi/egs/ac
!wget -c https://alphacephei.com/vosk-colab/vosk-model-small-en-us-0.15-compile-colab.tar.gz
!rm -rf vosk-model-small-en-us-0.15-compile-colab
!tar xf vosk-model-small-en-us-0.15-compile-colab.tar.gz

/content/kaldi/egs/ac
--2022-08-26 07:01:05--  https://alphacephei.com/vosk-colab/vosk-model-small-en-us-0.15-compile-colab.tar.gz
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59618100 (57M) [application/octet-stream]
Saving to: ‘vosk-model-small-en-us-0.15-compile-colab.tar.gz’

vosk-model-small-en 100%[===================>]  56.86M  10.4MB/s    in 6.7s    

2022-08-26 07:01:13 (8.49 MB/s) - ‘vosk-model-small-en-us-0.15-compile-colab.tar.gz’ saved [59618100/59618100]



In [3]:
%cd /content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab
!ls
!cat compile-graph.sh
!bash compile-graph.sh

/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab
compile-graph.sh  data_test  decode.sh	exp	      local  path.sh  steps
conf		  db	     dict.py	get_vocab.py  mfcc   RESULTS  utils
#!/bin/bash

set -x

. path.sh

pip3 install phonetisaurus

rm -rf data
rm -rf exp/tdnn/lgraph
rm -rf exp/tdnn/lgraph_orig

mkdir -p data/dict
cp db/phone/* data/dict
./dict.py > data/dict/lexicon.txt

python3 ./get_vocab.py > data/mix.vocab
ngramsymbols data/mix.vocab data/mix.syms
farcompilestrings --fst_type=compact --symbols=data/mix.syms --keep_symbols --unknown_symbol="[unk]" db/extra.txt data/extra.far
ngramcount --order=3 data/extra.far - |
    ngramprint --integers | grep -v "<unk>" | ngramread |
    ngramshrink --method=count_prune --count_pattern="3+:3" |
    ngrammake --method=witten_bell - data/extra.mod
gunzip -c db/en-50k-0.4-android.lm.gz | ngramread --renormalize_arpa --ARPA --symbols=data/mix.syms - data/en-us.mod
ngrammerge --method="bayes_model_merge" --normalize --alpha=0.9

In [4]:
!cat decode.sh
!bash decode.sh

#!/bin/bash

. path.sh

steps/make_mfcc.sh --nj 10 data_test/test_small exp/make_mfcc/test mfcc
steps/compute_cmvn_stats.sh data_test/test_small exp/make_mfcc/test mfcc
utils/fix_data_dir.sh data_test/test_small

steps/online/nnet2/extract_ivectors_online.sh --nj 4 \
          data_test/test_small exp/extractor \
          exp/ivectors_test

steps/nnet3/decode.sh --nj 4 \
          --acwt 1.0 --post-decode-acwt 10.0 \
          --online-ivector-dir exp/ivectors_test \
          exp/tdnn/graph_adapt data_test/test_small exp/tdnn/decode_test_adapt

steps/nnet3/decode.sh --nj 4 \
          --acwt 1.0 --post-decode-acwt 10.0 \
          --online-ivector-dir exp/ivectors_test \
          exp/tdnn/graph data_test/test_small exp/tdnn/decode_test

#steps/nnet3/decode_lookahead.sh --nj 4 \
#          --acwt 1.0 --post-decode-acwt 10.0 \
#          --online-ivector-dir exp/ivectors_test \
#          exp/tdnn/lgraph data_test/test_small exp/tdnn/decode_test_adapt
#steps/nnet3/decode_lookahead.sh 

In [5]:
!bash RESULTS


%WER 11.77 [ 107 / 909, 13 ins, 7 del, 87 sub ] exp/tdnn/decode_test/wer_7_1.0
%WER 0.22 [ 2 / 909, 0 ins, 1 del, 1 sub ] exp/tdnn/decode_test_adapt/wer_10_1.0


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cp -av "/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab" "/content/drive/MyDrive/AI"

'/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab' -> '/content/drive/MyDrive/AI/vosk-model-small-en-us-0.15-compile-colab'
'/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/data_test' -> '/content/drive/MyDrive/AI/vosk-model-small-en-us-0.15-compile-colab/data_test'
'/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/data_test/test_small' -> '/content/drive/MyDrive/AI/vosk-model-small-en-us-0.15-compile-colab/data_test/test_small'
'/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/data_test/test_small/cmvn.scp' -> '/content/drive/MyDrive/AI/vosk-model-small-en-us-0.15-compile-colab/data_test/test_small/cmvn.scp'
'/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/data_test/test_small/text.txt' -> '/content/drive/MyDrive/AI/vosk-model-small-en-us-0.15-compile-colab/data_test/test_small/text.txt'
'/content/kaldi/egs/ac/vosk-model-small-en-us-0.15-compile-colab/data_test/test_small/utt2dur' -> '/content/drive/MyDrive/A